# Thème 3: Aménagement cyclable/apaisement
**Correspondance cartographique :** densité point rouge_58 / vert_60 x type de voirie (Données Route 500) 
- Identifier la part des points rouges situées à moins de 50 mètres de voies / carrefours routières 
- La majorité des points rouges se situe sur des infrastructures routières complexes (voies structurantes / carrefour routier) 

> Hypothèse : Il faut transformer les infrastructures automobiles structurantes pour sécuriser la pratique des cyclistes

**Consignes supplémentaires :** Initialement l'idée était de faire ça avec les points uniques mais faire cela à partir des concentrations de points que tu as identifié sera sans doute plus pertinent.<br/>

identifier pour chaque cluster (vert / rouge) :
1. sur quels types de voirie il se situe (oui ou non présent sur un gros axe)
2. Et voir s'il y a une corrélation 

(on retrouve un lien entre cluster rouge & liaison à vocation régionale/nationale là où on a des infra de route les plus importantes les points les plus dangereux car ce n'est pas adaptée)
!= points verts --> liaisons locales (car c'est aménagés)


NB_VOIES --> danger car voie trop grande ??? 


Livrable : tableau + graphique + correlation ou pas

Cluster contient un carrefour ? --> typologie du carrefour lien entre la présence d'un carrefour et la présence d'un cluster ? 

## Python stuff

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
from s3_utils import S3Manager
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, fisher_exact

# Configs
_ = plt.style.use("ggplot")

In [2]:
s3_manager = S3Manager()
bucket = "fub-s3"

## 1 - Récupération et préparation des données

### 1.1 - Clusters en entrée

In [3]:
gdf_58 = gpd.read_file("temp/cluster_cat_58.geojson")
gdf_60 = gpd.read_file("temp/cluster_cat_60.geojson")

In [4]:
# Infos rapides sur les datasets 
print(gdf_58.crs if gdf_60.crs == gdf_58.crs else "Les CRS sont différents uniformiser les données")
gdf_58.head(3)

EPSG:4326


,cluster_id,uid,cat,uid_reponse,description,code_commune,nom_commune,nb_points,total_commune,taux_points,geometry
0,0,261dbaa4-2555-40b2-a627-cb7c0d947172,58,d9bfec3d-fef7-47c9-b949-60964467556f,,92049,Montrouge,89,554,16.06,"POLYGON ((2.31115 48.8156, 2.31105 48.81604, 2..."
1,1,8489e10a-8434-4d03-8161-a0606c67f44b,58,d9bfec3d-fef7-47c9-b949-60964467556f,,92049,Montrouge,109,554,19.68,"POLYGON ((2.31914 48.81765, 2.3185 48.8187, 2...."
2,2,8f6e0b77-2819-435e-a910-ceb2905e89ae,58,edf78d17-34e7-4c6a-879b-a3f298e67105,Grand échangeur,69199,Saint-Fons,24,37,64.86,"POLYGON ((4.85501 45.71655, 4.85466 45.71659, ..."


### 1.2 - Données d'enrichissement

In [5]:
# Téléchargement des fichiers depuis S3
# s3_manager.download_from_s3(bucket, "data/DFG/2025/data_geo/externe/3.B.COMMUNICATION_RESTREINTE.fgb", "temp/COMMUNICATION_RESTREINTE.fgb")
# s3_manager.download_from_s3(bucket, "data/DFG/2025/data_geo/externe/3.B.NOEUD_COMMUNE.fgb", "temp/NOEUD_COMMUNE.fgb")
s3_manager.download_from_s3(bucket, "data/DFG/2025/data_geo/externe/3.B.NOEUD_ROUTIER.fgb", "temp/NOEUD_ROUTIER.fgb")
# s3_manager.download_from_s3(bucket, "data/DFG/2025/data_geo/externe/3.B.TRONCON_ROUTE.fgb", "temp/TRONCON_ROUTE.fgb")

✅ Fichier téléchargé depuis S3 : temp/NOEUD_ROUTIER.fgb


In [6]:
# communication_restreinte = gpd.read_file("temp/COMMUNICATION_RESTREINTE.fgb") # ignoré
# noeud_commune = gpd.read_file("temp/NOEUD_COMMUNE.fgb") # doublon spatial avec noeud_routier
# troncon_route = gpd.read_file("temp/TRONCON_ROUTE.fgb")
noeud_routier = gpd.read_file("temp/NOEUD_ROUTIER.fgb")

In [7]:
noeud_routier.head()

,ID_RTE500,NATURE,geometry
0,192745,Changement d'attribut,POINT (1222200.161 6052904.435)
1,192742,Carrefour simple,POINT (1222863.857 6053870.408)
2,192737,Changement d'attribut,POINT (1222546.693 6055511.21)
3,192738,Carrefour simple,POINT (1221728.746 6054722.377)
4,192739,Carrefour simple,POINT (1216993.712 6054744.081)


#### 2.1.1 - Statistiques globales 